# Analyse, classification et indexation des données: feuille 6
### Classifieur bayésien

Dans la première partie de ce TD on s’appuie sur l’exemple du cours qui consiste à séparer des poissons en deux classes : bars et saumons. La classification sera faite à partir d’une seule caractéristique (la longueur) dans un premier temps, puis à partir de deux caractéristiques (longueur et brillance). En d’autres termes, le descripteur sera d’abord de dimension 1 puis de dimension 2.

#### Données
Dans cet exercice, nous allons travailler avec un fichier (<code>salmon_seabass.csv</code>) contenant des informations sur la longueur et la brillance de deux classes de poissons : les saumons et les bars.
Le fichier est disponible à l'adresse : 

https://www.labri.fr/perso/zemmari/datasets/salmon_seabass.csv

1- Chargez le fichier, et notez que la colonne <code>species</code> indique la catégorie du poisson : 

    - 0 : il s'agit d'un saumon 
    - 1 : il s'agit d'un bar


In [ ]:
import pandas as pa

In [ ]:
### CORRECTION
data = pa.read_csv('https://www.labri.fr/perso/zemmari/datasets/salmon_seabass.csv', delimiter=';', decimal='.')
data.head()

In [ ]:
### CORRECTION
print(data.species.unique())
# idem à data["species"].unique()

2- Affichez le nombre de valeurs fournies pour chacune des deux classes. On les nommera <code>sizeVTSaumon</code>
et <code>sizeVTBar</code>. 

In [ ]:
### CORRECTION
sizeVTSaumon = len(data[data.species==0]) # ou data[data.species==0].shape[0]
sizeVTBar  = len(data[data.species==1])
print(sizeVTSaumon)
print(sizeVTBar)

### Exercice 1. classifieur avec une seule dimension

### 1.0 Préalable
Dans un premier temps, nous allons considérer un seul descripteur : la longueur <code>width</code> des poissons. 

a- Mélanger les données.

In [ ]:
### CORRECTION
data = data.sample(frac=1)
data.head()

b- Créer deux variables $X$ et $y$ respectivement la longueur du poisson et son espèce

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### CORRECTION
X = data.width
y = data.species

c-  Découper les données en deux parties : 80% pour l'entrainement et 20% pour le test.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
### CORRECTION
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=.2)

#### 1.1. Classifieur ML

a- Calculer les estimateurs non biaisés de la moyenne et de l'écart type pour chacune des espèces de poisson. Attention, il faut travailler sur la portion d'entraînement du dataset.  


In [ ]:
import numpy as np

In [ ]:
### CORRECTION
X_saumon = X_train[y_train==0]
X_bar = X_train[y_train==1]

mean_s = np.mean(X_saumon)
mean_b = np.mean(X_bar)

sigma_s = np.std(X_saumon)
sigma_b = np.std(X_bar)

b- Dessiner les courbes des lois normales correspondant à la fonction de maximum de vraisemblance. Indiquer la frontière de décision. Quelle est l'espèce prédite d'un poisson de longueur 12 ?

In [ ]:
import scipy.stats

In [ ]:
### CORRECTION
x_min, x_max = 0, 20
x = np.linspace(x_min, x_max, 100)

y_s = scipy.stats.norm.pdf(x,mean_s,sigma_s)
y_b = scipy.stats.norm.pdf(x,mean_b,sigma_b)

plt.plot(x,y_s, color='r', label='saumon')
plt.plot(x,y_b, color='g', label='bar')


plt.title('f(l | classe)',fontsize=10)

plt.xlabel('length')
plt.legend(loc='upper right')
plt.grid()
plt.show()


### CORRECTION
La frontière de décision est située à ~10. 

$$f(l=12 \mid saumon) < f(l=12 \mid bar)$$

$\Rightarrow$ le poisson est un bar.

c- Evaluer la qualité du modèle entrainé.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
### CORRECTION
X_test = list(X_test)

In [ ]:
### CORRECTION
y_pred = []
for i in range(len(X_test)):
    if scipy.stats.norm.pdf(X_test[i],mean_s,sigma_s) > scipy.stats.norm.pdf(X_test[i],mean_b,sigma_b):
        y_pred.append(0)
    else:
        y_pred.append(1)

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(acc))

#### 1.2. Classifieur MAP

a- Ecrire une fonction <code>classify()</code> permettant également de classifier les poissons mais en utilisant cette fois un classifieur a posteriori.

In [ ]:
### CORRECTION
def classify(item): 
    #Probabilités apriori:
    p_saumon = len(X_saumon)/len(X_train)
    p_bar = len(X_bar)/len(X_train)
    #print(p_saumon) # environ 2/3
    #print(p_bar) # environ 1/3
    ps = p_saumon * scipy.stats.norm.pdf(item,mean_s,sigma_s)
    pb = p_bar * scipy.stats.norm.pdf(item,mean_b,sigma_b)
    if ps > pb:
        return 0
    return 1

print("classe prédite pour item 0 : ", classify(X_test[0]))

b- Evaluer ce classifieur 

In [ ]:
### CORRECTION
y_pred = [classify(X_test[i]) for i in range(len(X_test))]

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(acc))

### CORRECTION
On classifie plus facilement un poisson dans la classe la plus représentée (saumon), donc on fait moins d'erreur sur les saumons. Par contre on va plus souvent classifier un bar comme saumon.


### Exercice 2. Classifieur avec deux descripteurs 

Dans cet exercice, nous allons considérer le dataset dans sa globalité. Nous allons donc baser notre classification sur les deux descripteurs <code>width</code> et <code>lightness</code>.

Reprogrammer et tester les deux classifieurs vus dans l'exercice 1. Quel est l'impact du rajout du nouveau descripteur ?

In [ ]:
### CORRECTION
data = pa.read_csv('https://www.labri.fr/perso/zemmari/datasets/salmon_seabass.csv', delimiter=';', decimal='.')

X = data[['width', 'lightness']]
y = data.species

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=.2)

X_saumon = X_train[y_train==0]
X_bar = X_train[y_train==1]

mean_s = np.mean(X_saumon, axis=0)
mean_b = np.mean(X_bar, axis=0)

sigma_s = np.std(X_saumon)
sigma_b = np.std(X_bar)
print("mean_s : \n", mean_s)
print("sigma_s : \n", sigma_s)


In [ ]:
### CORRECTION
#Classifieur ML :
def ml_classify(item):
    #print(item)
    f_s = scipy.stats.norm.pdf(item[0],mean_s[0],sigma_s[0]) * scipy.stats.norm.pdf(item[0],mean_s[1],sigma_s[1])
    f_b = scipy.stats.norm.pdf(item[1],mean_b[0],sigma_b[0]) * scipy.stats.norm.pdf(item[1],mean_b[1],sigma_b[1])
    if f_s > f_b:
        return 0
    return 1

In [ ]:
### CORRECTION
X_test = np.array(X_test)
ml_classify(X_test[0])

In [ ]:
### CORRECTION
y_pred = [ml_classify(X_test[i]) for i in range(len(X_test))]

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(acc))

In [ ]:
### CORRECTION
#Classifieur MAP :
def map_classify(item):
    #print(item)
    #Probabilités apriori:
    p_saumon = len(X_saumon)/len(X_train)
    p_bar = len(X_bar)/len(X_train)
    f_s = scipy.stats.norm.pdf(item[0],mean_s[0],sigma_s[0]) * scipy.stats.norm.pdf(item[0],mean_s[1],sigma_s[1])
    f_b = scipy.stats.norm.pdf(item[1],mean_b[0],sigma_b[0]) * scipy.stats.norm.pdf(item[1],mean_b[1],sigma_b[1])
    ps = p_saumon * f_s
    pb = p_bar * f_b
    if ps > pb:
        return 0
    return 1

In [ ]:
### CORRECTION
y_pred = [map_classify(X_test[i]) for i in range(len(X_test))]

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(acc))

### Exercice 3. Bayes avec <code>sklearn</code>

Comparer vos résultats avec ceux obtenus avec un classifieur de Bayes de la bibliothèque <code>sklearn</code>.

In [ ]:
### CORRECTION
data = pa.read_csv('https://www.labri.fr/perso/zemmari/datasets/salmon_seabass.csv', delimiter=';', decimal='.')

X = data[['width', 'lightness']]
y = data.species

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=.2)


In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
### CORRECTION
nb = GaussianNB()
nb.fit(X_train, y_train)

In [ ]:
### CORRECTION
y_pred = nb.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)

acc = accuracy_score(y_test, y_pred)
print('{:.2%}'.format(acc))

In [ ]:
### CORRECTION
nb.get_params()
